In [ ]:
! pip install -q face_recognition
! pip install -q opencv-python-headless
! apt-get update && apt-get install -y \
    libgl1-mesa-glx \
    libglib2.0-0

In [ ]:
import os
import sqlite3
import numpy as np
import cv2
import face_recognition
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import ipywidgets as widgets

class FaceRecognitionSystem:
    def __init__(self, db_path='/content/user_faces.db'):
        """
        Initialize the Face Recognition System for Google Colab

        Args:
            db_path (str): Path to the SQLite database file
        """
        self.db_path = db_path
        self.conn = None
        self.cursor = None
        self._create_database()

    def _create_database(self):
        """
        Create SQLite database and users table if not exists
        """
        try:
            self.conn = sqlite3.connect(self.db_path)
            self.cursor = self.conn.cursor()

            # Create table to store user face encodings
            self.cursor.execute('''
                CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    username TEXT UNIQUE NOT NULL,
                    face_encoding BLOB NOT NULL
                )
            ''')
            self.conn.commit()
        except sqlite3.Error as e:
            print(f"Database error: {e}")

    def _js_cv_stream(self):
        """
        JavaScript for capturing video stream in Colab with improved shutdown
        """
        js = Javascript('''
        var video;
        var div = null;
        var stream;
        var captureCanvas;
        var imgElement;

        var pendingResolve = null;
        var shutdown = false;

        function removeDom() {
            if (stream) {
                stream.getTracks().forEach(track => track.stop());
            }
            if (video) video.remove();
            if (div) div.remove();
            video = null;
            div = null;
            stream = null;
            imgElement = null;
            captureCanvas = null;
            pendingResolve = null;
            shutdown = true;
        }

        function onAnimationFrame() {
            if (!shutdown) {
                window.requestAnimationFrame(onAnimationFrame);
            }
            if (pendingResolve) {
                var result = "";
                if (!shutdown) {
                    captureCanvas.getContext('2d').drawImage(video, 0, 0);
                    result = captureCanvas.toDataURL('image/jpeg', 0.8)
                }
                var lp = pendingResolve;
                pendingResolve = null;
                lp(result);
            }
        }

        async function createDom() {
            if (div !== null) {
                return stream;
            }

            div = document.createElement('div');
            div.style.border = '2px solid black';
            div.style.padding = '3px';
            div.style.width = '100%';
            div.style.maxWidth = '600px';
            document.body.appendChild(div);

            video = document.createElement('video');
            video.style.display = 'block';
            video.width = div.clientWidth - 6;
            video.setAttribute('playsinline', '');
            video.onclick = () => {
                shutdown = true;
                removeDom();
            };
            stream = await navigator.mediaDevices.getUserMedia(
                {video: { facingMode: "user" }});
            div.appendChild(video);

            imgElement = document.createElement('img');
            imgElement.style.position = 'absolute';
            imgElement.style.zIndex = 1;
            imgElement.onclick = () => {
                shutdown = true;
                removeDom();
            };
            div.appendChild(imgElement);

            const instruction = document.createElement('div');
            instruction.innerHTML = 'Click anywhere to stop video capture';
            instruction.style.color = 'red';
            instruction.style.fontWeight = 'bold';
            div.appendChild(instruction);
            instruction.onclick = () => {
                shutdown = true;
                removeDom();
            };

            video.srcObject = stream;
            await video.play();

            captureCanvas = document.createElement('canvas');
            captureCanvas.width = video.videoWidth;
            captureCanvas.height = video.videoHeight;
            window.requestAnimationFrame(onAnimationFrame);

            return stream;
        }

        async function stream_frame(label, imgData) {
            if (shutdown) {
                removeDom();
                shutdown = true;
                return '';
            }

            stream = await createDom();

            if (imgData != "") {
                var videoRect = video.getBoundingClientRect();
                imgElement.style.top = videoRect.top + "px";
                imgElement.style.left = videoRect.left + "px";
                imgElement.style.width = videoRect.width + "px";
                imgElement.style.height = videoRect.height + "px";
                imgElement.src = imgData;
            }

            var result = await new Promise(function(resolve, reject) {
                pendingResolve = resolve;
            });
            shutdown = false;

            return result;
        }
        ''')
        display(js)

    def register_user(self, username):
        """
        Register a new user in Colab using JavaScript video capture

        Args:
            username (str): Username for the new user
        """
        # Check if username already exists
        self.cursor.execute("SELECT * FROM users WHERE username = ?", (username,))
        if self.cursor.fetchone():
            print(f"User {username} already exists!")
            return False

        # Setup JavaScript video streaming
        self._js_cv_stream()

        print(f"Registering user: {username}")
        print("Look at the camera. Ensure your face is clearly visible.")

        js_reply = eval_js('stream_frame("", "")')

        if not js_reply:
            print("Failed to capture image.")
            return False

        # Convert base64 to numpy array
        image_bytes = b64decode(js_reply.split(',')[1])
        jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
        frame = cv2.imdecode(jpg_as_np, flags=1)

        # Convert to RGB for face recognition
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect faces with increased upsampling for better detection
        face_locations = face_recognition.face_locations(
            rgb_frame,
            number_of_times_to_upsample=1,  
            model="hog"  
        )
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        if not face_encodings:
            print("No face detected. Please try again.")
            return False

        # Annotate frame with face detection
        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        # Convert annotated frame back to base64 for display
        _, buffer = cv2.imencode('.jpg', frame)
        jpg_as_text = f'data:image/jpeg;base64,{b64encode(buffer).decode()}'

        # Show annotated frame
        eval_js(f'stream_frame("Face Detected", "{jpg_as_text}")')

        # Take the first detected face
        face_encoding = face_encodings[0]

        # Convert encoding to binary for storage
        encoding_bytes = face_encoding.tobytes()

        try:
            self.cursor.execute(
                "INSERT INTO users (username, face_encoding) VALUES (?, ?)",
                (username, encoding_bytes)
            )
            self.conn.commit()
            print(f"User {username} registered successfully!")
            return True
        except sqlite3.IntegrityError:
            print("Registration failed. Username might already exist.")
            return False
        except Exception as e:
            print(f"Error during registration: {e}")
            return False

    def recognize_user(self):
        """
        Recognize users using Colab's JavaScript video capture
        """
        self.cursor.execute("SELECT username, face_encoding FROM users")
        users = self.cursor.fetchall()

        if not users:
            print("No users registered. Please register first.")
            return None

        known_encodings = []
        known_usernames = []

        for username, encoding_bytes in users:
            # Convert binary back to numpy array
            face_encoding = np.frombuffer(encoding_bytes, dtype=np.float64)
            known_encodings.append(face_encoding)
            known_usernames.append(username)

        # Setup JavaScript video streaming
        self._js_cv_stream()

        print("Face Recognition Started. Look at the camera.")

        while True:
            js_reply = eval_js('stream_frame("Face Recognition", "")')
            if not js_reply:
                break

            image_bytes = b64decode(js_reply.split(',')[1])
            jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
            frame = cv2.imdecode(jpg_as_np, flags=1)

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            face_locations = face_recognition.face_locations(
                rgb_frame,
                number_of_times_to_upsample=1, 
                model="hog" 
            )
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

          
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
          
                matches = face_recognition.compare_faces(known_encodings, face_encoding)
                name = "Unknown"  
                match_percentage = 0

                face_distances = face_recognition.face_distance(known_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)

                if matches[best_match_index]:
                    name = known_usernames[best_match_index]
                    # Calculate match percentage (inverse of face distance)
                    match_percentage = round((1 - face_distances[best_match_index]) * 100, 2)

                # Draw rectangle and label
                color = (0, 0, 255) if name == "Unknown" else (0, 255, 0)
                cv2.rectangle(frame, (left, top), (right, bottom), color, 2)

                # Display name and match percentage
                label = f"{name} ({match_percentage}%)" if name != "Unknown" else "Unknown"
                cv2.putText(frame, label, (left, top-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            _, buffer = cv2.imencode('.jpg', frame)
            jpg_as_text = f'data:image/jpeg;base64,{b64encode(buffer).decode()}'

            if 'name' in locals():
                eval_js(f'stream_frame("Recognized: {name}", "{jpg_as_text}")')

    def list_users(self):
        """
        List all registered users
        """
        self.cursor.execute("SELECT username FROM users")
        users = self.cursor.fetchall()

        if not users:
            print("No users registered.")
        else:
            print("Registered Users:")
            for user in users:
                print(user[0])

    def delete_user(self, username):
        """
        Delete a user from the database

        Args:
            username (str): Username to delete
        """
        self.cursor.execute("DELETE FROM users WHERE username = ?", (username,))
        if self.cursor.rowcount > 0:
            self.conn.commit()
            print(f"User {username} deleted successfully.")
        else:
            print(f"User {username} not found.")

    def close_connection(self):
        """
        Close database connection
        """
        if self.conn:
            self.conn.close()

def main():

    face_system = FaceRecognitionSystem()

    def register_action(b):
        username = username_input.value
        if username:
            face_system.register_user(username)
        else:
            print("Please enter a username")

    def recognize_action(b):
        face_system.recognize_user()

    def list_users_action(b):
        face_system.list_users()

    def delete_user_action(b):
        username = delete_username_input.value
        if username:
            face_system.delete_user(username)
        else:
            print("Please enter a username to delete")

    # interactive widgets
    username_input = widgets.Text(description='Username:')
    register_button = widgets.Button(description='Register')
    register_button.on_click(register_action)

    delete_username_input = widgets.Text(description='Username:')
    delete_button = widgets.Button(description='Delete User')
    delete_button.on_click(delete_user_action)

    recognize_button = widgets.Button(description='Recognize Users')
    recognize_button.on_click(recognize_action)

    list_users_button = widgets.Button(description='List Users')
    list_users_button.on_click(list_users_action)

    # Display widgets
    display(widgets.VBox([
        widgets.HTML("<h2>Face Recognition System</h2>"),
        widgets.HTML("<h3>Register User</h3>"),
        username_input,
        register_button,
        widgets.HTML("<h3>Delete User</h3>"),
        delete_username_input,
        delete_button,
        widgets.HTML("<h3>Other Actions</h3>"),
        recognize_button,
        list_users_button
    ]))

main()